In [45]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight
from sklearn.metrics import roc_auc_score, accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

ID_EMBED_DIM = 64                   # detail van kaarten 
LVL_EMBED_DIM = 8                   # detail van levels 
HIDDEN_DIMS = [512, 256]             # aantal neuronen in 1ste en 2de laag
DROPOUT = 0.3                       # willekeurige neuronen 
BATCH_SIZE = 4096                  # aantal bekeken wedstrijden per keer 
EPOCHS = 10                        # aantal herhalingen van dataset
LR = 0.001                          # diepgang van het model
MODEL_DIR = "pytorch_card_model"    # getrainde model     

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # gebruik GPU(nvidia) indien mogelijk anders cpu 
print("Using device:", device)
os.makedirs(MODEL_DIR, exist_ok=True)

Using device: cuda


In [46]:
import pandas as pd
import itertools

# 1. Zet hier al je bestandsnamen in een lijst
bestanden_lijst = [
    "BattlesStaging_01012021_WL_tagged.csv",
    "BattlesStaging_01022021_WL_tagged.csv", 
    "BattlesStaging_01032021_WL_tagged.csv", 
    "BattlesStaging_01042021_WL_tagged.csv"
]

df = pd.read_csv("BattlesStaging_01012021_WL_tagged.csv")

In [47]:
import json
# -------------------------------------------------------------------
# 2. Identify card ID and level columns
# -------------------------------------------------------------------
winner_id_cols   = [f"winner.card{i}.id" for i in range(1, 9)]                              # sla kaarten van winnaar op 
winner_lvl_cols  = [f"winner.card{i}.level" for i in range(1, 9)]                           # sla level op van kaarten winnaar 
loser_id_cols    = [f"loser.card{i}.id" for i in range(1, 9)]                               # sla kaarten van verliezer op
loser_lvl_cols   = [f"loser.card{i}.level" for i in range(1, 9)]                            # sla kaartlevels van verliezer op 

all_card_id_cols = winner_id_cols + loser_id_cols                                           # sla alle gebruikte kaarten in de match op 

# -------------------------------------------------------------------
# 3. Extract unique card IDs across all matches and remap to integers
# -------------------------------------------------------------------
unique_card_ids = pd.unique(df[all_card_id_cols].values.ravel())                            # geef elke kaart een unieke ID

id_to_compact = {orig_id: idx for idx, orig_id in enumerate(unique_card_ids)}               

# Save mapping
with open("card_id_mapping.json", "w") as f:                                                # hou vertaling van kaart -> unieke id bij 
    json.dump({str(k): v for k, v in id_to_compact.items()}, f)

# Apply mapping
for col in all_card_id_cols:                                                                # vervangt elke kaart door zijn unieke ID 
    df[col] = df[col].map(id_to_compact)

# -------------------------------------------------------------------
# 4. Subset dataset to only card id + card level + winner/loser tags
# -------------------------------------------------------------------
df_small = df[winner_id_cols + winner_lvl_cols + loser_id_cols + loser_lvl_cols + ["winner.tag", "loser.tag"]].copy()           #verwijderd nutteloze data

# -------------------------------------------------------------------
# 5. Randomize who is player1 and player2
# -------------------------------------------------------------------
flip = np.random.randint(0, 2, size=len(df_small))  # 0 = swap, 1 = normal

player1_ids   = np.where(flip[:, None] == 1, df_small[winner_id_cols].values,  df_small[loser_id_cols].values)      # winnaar staat eerst 
player1_lvls  = np.where(flip[:, None] == 1, df_small[winner_lvl_cols].values, df_small[loser_lvl_cols].values)

player2_ids   = np.where(flip[:, None] == 1, df_small[loser_id_cols].values,  df_small[winner_id_cols].values)      # verliezer staat eerst 
player2_lvls  = np.where(flip[:, None] == 1, df_small[loser_lvl_cols].values, df_small[winner_lvl_cols].values)

# Determine winner based on flip
winner_player = np.where(flip == 1, "player1", "player2")                                                           # houdt bij welke speler eerst stond

# -------------------------------------------------------------------
# 6. Build final cleaned DataFrame
# -------------------------------------------------------------------
final_data = {}

# Player 1
for i in range(1, 9):
    final_data[f"player1.card{i}.id"]    = player1_ids[:, i-1]
    final_data[f"player1.card{i}.level"] = player1_lvls[:, i-1]

# Player 2
for i in range(1, 9):
    final_data[f"player2.card{i}.id"]    = player2_ids[:, i-1]
    final_data[f"player2.card{i}.level"] = player2_lvls[:, i-1]

# Winner label
final_data["winner_player"] = winner_player

# Optional: keep original player tags
final_data["winner.tag"] = df_small["winner.tag"].values
final_data["loser.tag"]  = df_small["loser.tag"].values

df_clean = pd.DataFrame(final_data)

# -------------------------------------------------------------------
# 7. Save final dataset
# -------------------------------------------------------------------
df_clean.to_csv("matches_clean_randomized.csv", index=False)

df_clean.head()


,player1.card1.id,player1.card1.level,player1.card2.id,player1.card2.level,player1.card3.id,player1.card3.level,player1.card4.id,player1.card4.level,player1.card5.id,player1.card5.level,...,player2.card5.level,player2.card6.id,player2.card6.level,player2.card7.id,player2.card7.level,player2.card8.id,player2.card8.level,winner_player,winner.tag,loser.tag
0,0,13,1,13,2,13,3,13,4,13,...,13,12,13,13,13,14,13,player1,#PVLPJP2Y,#PLYJVUQY2
1,15,13,16,12,17,12,18,12,19,13,...,13,26,13,12,13,27,13,player1,#8PRLRYYCV,#92VG2CPY
2,23,13,33,13,34,13,35,13,1,13,...,13,31,13,14,13,32,13,player2,#2G8LQRCG,#2PCUY9U80
3,19,11,36,9,33,11,37,10,21,10,...,10,40,10,26,10,41,10,player1,#Y9QL09VGV,#9GJJGYL8P
4,42,13,5,13,22,13,16,13,47,13,...,13,45,13,18,13,46,13,player2,#9RRYG9P9U,#80J0LUCP8


In [48]:
# Cell 2 - Load CSV & inspect
CSV_PATH = "matches_clean_randomized.csv"  # replace with your file path if different
df = pd.read_csv(CSV_PATH)
print("rows,cols:", df.shape)
df.head()


rows,cols: (2823527, 35)


,player1.card1.id,player1.card1.level,player1.card2.id,player1.card2.level,player1.card3.id,player1.card3.level,player1.card4.id,player1.card4.level,player1.card5.id,player1.card5.level,...,player2.card5.level,player2.card6.id,player2.card6.level,player2.card7.id,player2.card7.level,player2.card8.id,player2.card8.level,winner_player,winner.tag,loser.tag
0,0,13,1,13,2,13,3,13,4,13,...,13,12,13,13,13,14,13,player1,#PVLPJP2Y,#PLYJVUQY2
1,15,13,16,12,17,12,18,12,19,13,...,13,26,13,12,13,27,13,player1,#8PRLRYYCV,#92VG2CPY
2,23,13,33,13,34,13,35,13,1,13,...,13,31,13,14,13,32,13,player2,#2G8LQRCG,#2PCUY9U80
3,19,11,36,9,33,11,37,10,21,10,...,10,40,10,26,10,41,10,player1,#Y9QL09VGV,#9GJJGYL8P
4,42,13,5,13,22,13,16,13,47,13,...,13,45,13,18,13,46,13,player2,#9RRYG9P9U,#80J0LUCP8


In [49]:
# Cell 3 - Extract card id & level arrays and labels

p1_id_cols = [f"player1.card{i}.id" for i in range(1,9)]                        # zoek naar kaartids van speler 1
p1_lvl_cols = [f"player1.card{i}.level" for i in range(1,9)]                    # zoek naar levels van speler 1 
p2_id_cols = [f"player2.card{i}.id" for i in range(1,9)]                        # zoek naar kaartids van speler 2 
p2_lvl_cols = [f"player2.card{i}.level" for i in range(1,9)]                    # zoek naar levels van speler 2 

# Safety check: ensure columns exist
missing = [c for c in (p1_id_cols+p1_lvl_cols+p2_id_cols+p2_lvl_cols+["winner_player"]) if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

# X arrays
X_p1_ids  = df[p1_id_cols].fillna(-1).astype(int).values  # shape (N,8)  controle en verbetering van data 
X_p1_lvls = df[p1_lvl_cols].fillna(0).astype(int).values
X_p2_ids  = df[p2_id_cols].fillna(-1).astype(int).values
X_p2_lvls = df[p2_lvl_cols].fillna(0).astype(int).values

# Label: 1 if player1 won else 0
y = (df["winner_player"] == "player1").astype(int).values           # waarde die het algoritme moet voorspellen

print("N examples:", len(y))                        
print("Example ids p1:", X_p1_ids[0])
print("Example lvls p1:", X_p1_lvls[0])

# Determine vocab sizes
max_card_id = int(np.max(np.concatenate([X_p1_ids.ravel(), X_p2_ids.ravel()])))             # zoekt hoogste kaartnummer
if max_card_id < 0:     
    raise ValueError("All card ids are negative or empty.")                                 # error als er minder dan 0 kaarten zijn
vocab_size = max_card_id + 1  # assuming ids start at 0                                     
max_level = int(np.max(np.concatenate([X_p1_lvls.ravel(), X_p2_lvls.ravel()])))             # zoekt hoogste kaartlevel
print("vocab_size (max id+1) =", vocab_size, "max_level =", max_level)


N examples: 2823527
Example ids p1: [0 1 2 3 4 5 6 7]
Example lvls p1: [13 13 13 13 13 13 13 13]
vocab_size (max id+1) = 102 max_level = 13


In [50]:
# Cell 4 - Train/Val split and class weights
X = {                                           # alle variabelen verzameld in x 
    "p1_ids": X_p1_ids,
    "p1_lvls": X_p1_lvls,
    "p2_ids": X_p2_ids,
    "p2_lvls": X_p2_lvls
}

Xw_train, Xw_val, Xl_train, Xl_val, y_train, y_val = train_test_split(                          # ongebruike code die de levels niet gebruikt
    X["p1_ids"], X["p2_ids"], y, test_size=0.1, stratify=y, random_state=42
)
# But we split p1 ids and p2 ids together via indexes; better create indices:
# We'll create index split explicitly to keep lvls aligned

from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)            # 0.1/1 is van de data is voor tests en de rest voor training 
train_idx, val_idx = next(sss.split(X["p1_ids"], y))                                # maak lijst om te testen en een lijst om te trainen
Xw_train_ids  = X["p1_ids"][train_idx]                                              # maakt lijst met trainingdata
Xw_train_lvls = X["p1_lvls"][train_idx]
Xl_train_ids  = X["p2_ids"][train_idx]
Xl_train_lvls = X["p2_lvls"][train_idx]
y_train = y[train_idx]

Xw_val_ids  = X["p1_ids"][val_idx]                                                  # maakt lijst met testdata
Xw_val_lvls = X["p1_lvls"][val_idx]
Xl_val_ids  = X["p2_ids"][val_idx]
Xl_val_lvls = X["p2_lvls"][val_idx]
y_val = y[val_idx]

print("Train/Val sizes:", len(y_train), len(y_val))

# class weights for loss (optional)
class_weights = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)             # berekenen van reward system (moet ong gelijk zijn voor 1 en 2)
class_weights = torch.tensor(class_weights, dtype=torch.float32, device=device)
print("class_weights:", class_weights.cpu().numpy())


Train/Val sizes: 2541174 282353
class_weights: [1.0001913 0.9998088]


In [51]:
# Cell 5 - Dataset and DataLoader
class MatchDataset(Dataset):
    def __init__(self, p1_ids, p1_lvls, p2_ids, p2_lvls, labels):               # kaarten ids en levels zijn gehele getallen en uitkomst is een kommagetal
        self.p1_ids = torch.tensor(p1_ids, dtype=torch.long)
        self.p1_lvls = torch.tensor(p1_lvls, dtype=torch.long)
        self.p2_ids = torch.tensor(p2_ids, dtype=torch.long)
        self.p2_lvls = torch.tensor(p2_lvls, dtype=torch.long)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):                                                          # geeft aantal voorbeelden 
        return len(self.labels)

    def __getitem__(self, idx):                                                 # geeft waardes van elke wedstrijd op basis van wedstrijdID
        return (self.p1_ids[idx], self.p1_lvls[idx],
                self.p2_ids[idx], self.p2_lvls[idx],
                self.labels[idx])

train_ds = MatchDataset(Xw_train_ids, Xw_train_lvls, Xl_train_ids, Xl_train_lvls, y_train)      # zet alle trainingdata samen 
val_ds   = MatchDataset(Xw_val_ids, Xw_val_lvls, Xl_val_ids, Xl_val_lvls, y_val)                # zet alle validatiedata samen

# Zet workers op 0. Dit werkt ALTIJD, maar is ietsjes trager.
train_loader = DataLoader(
    train_ds, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    drop_last=False, 
    num_workers=0,         
    pin_memory=True         # Dit mag wel aan blijven voor je GPU
    # persistent_workers haal je weg, dat werkt niet met 0 workers
)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)                                 

print("Batches train:", len(train_loader), "val:", len(val_loader))


Batches train: 621 val: 69


In [52]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CounterAwareNet(nn.Module):
    def __init__(self, vocab_size, max_level, id_emb_dim=64, lvl_emb_dim=8, hidden_dims=[512, 256], dropout=0.2, num_heads=4):
        super().__init__()
        
        # 1. Embeddings (Dezelfde als eerst)
        self.id_embed = nn.Embedding(vocab_size, id_emb_dim)
        self.lvl_embed = nn.Embedding(max_level + 1, lvl_emb_dim)
        self.card_dim = id_emb_dim + lvl_emb_dim
        
        # 2. Transformer (Om de synergie BINNEN een deck te leren)
        # We verwerken P1 en P2 nu APART, zodat we een zuivere representatie van elk deck krijgen
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.card_dim, nhead=num_heads, batch_first=True, dropout=dropout)
        self.deck_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        
        # 3. DE MAGISCHE INTERACTIE LAAG (Het Counter Mechanisme)
        # We berekenen hoe elke kaart van P1 reageert op elke kaart van P2
        # Dit resulteert in een 8x8 matrix per gevecht
        
        # 4. De Classifier (MLP)
        # Input is: 
        # - Samenvatting P1 Deck (Global Average)
        # - Samenvatting P2 Deck (Global Average)
        # - De Counter Matrix (afgevlakt: 8*8 = 64 interacties)
        
        flattened_matrix_dim = 8 * 8 
        input_dim = (self.card_dim * 2) + flattened_matrix_dim 
        
        mlp_layers = []
        inp = input_dim
        for h in hidden_dims:
            mlp_layers.append(nn.Linear(inp, h))
            mlp_layers.append(nn.ReLU())
            mlp_layers.append(nn.Dropout(dropout))
            inp = h
        mlp_layers.append(nn.Linear(inp, 1))
        self.mlp = nn.Sequential(*mlp_layers)

    def forward(self, p1_ids, p1_lvls, p2_ids, p2_lvls):
        # A. Embeddings maken
        p1_emb = torch.cat([self.id_embed(p1_ids), self.lvl_embed(p1_lvls)], dim=-1) # [Batch, 8, Dim]
        p2_emb = torch.cat([self.id_embed(p2_ids), self.lvl_embed(p2_lvls)], dim=-1) # [Batch, 8, Dim]
        
        # B. Transformer: Leer de strategie van het deck zelf
        p1_encoded = self.deck_encoder(p1_emb) # [Batch, 8, Dim]
        p2_encoded = self.deck_encoder(p2_emb) # [Batch, 8, Dim]
        
        # C. INTERACTIE MATRIX (Dot Product Attention)
        # Hier dwingen we het model om elke kaart met elke vijandelijke kaart te vergelijken
        # matrix[i, j] vertelt ons hoe sterk Kaart i van P1 is tegen Kaart j van P2
        # We gebruiken Batch Matrix Multiplication (bmm)
        # [Batch, 8, Dim] x [Batch, Dim, 8] -> [Batch, 8, 8]
        interaction_matrix = torch.bmm(p1_encoded, p2_encoded.transpose(1, 2))
        
        # Normaliseren helpt bij het leren (zodat waarden niet exploderen)
        interaction_matrix = interaction_matrix / (self.card_dim ** 0.5)
        interaction_features = interaction_matrix.flatten(start_dim=1) # [Batch, 64]
        
        # D. Samenvatting per deck (Global Average Pooling)
        p1_summary = p1_encoded.mean(dim=1) # [Batch, Dim]
        p2_summary = p2_encoded.mean(dim=1) # [Batch, Dim]
        
        # E. Alles samenvoegen en voorspellen
        combined = torch.cat([p1_summary, p2_summary, interaction_features], dim=1)
        logit = self.mlp(combined).squeeze(1)
        
        return logit

# Model aanmaken met je krachtige GPU settings
model = TransformerCardNet(
    vocab_size=vocab_size, 
    max_level=max_level,
    id_emb_dim=ID_EMBED_DIM,    # 64
    lvl_emb_dim=LVL_EMBED_DIM,  # 8
    hidden_dims=HIDDEN_DIMS,    # [512, 256]
    dropout=DROPOUT,
    num_heads=4                 # Aantal 'aandachts-punten' per kaart
).to(device)

print(model)

 


TransformerCardNet(
  (id_embed): Embedding(102, 64)
  (lvl_embed): Embedding(14, 8)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=72, out_features=72, bias=True)
        )
        (linear1): Linear(in_features=72, out_features=2048, bias=True)
        (dropout): Dropout(p=0.3, inplace=False)
        (linear2): Linear(in_features=2048, out_features=72, bias=True)
        (norm1): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((72,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.3, inplace=False)
        (dropout2): Dropout(p=0.3, inplace=False)
      )
    )
  )
  (mlp): Sequential(
    (0): Linear(in_features=1152, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): 

In [53]:
# Cell 7 - Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
# Use BCEWithLogitsLoss and pass pos_weight for imbalance (pos_weight = weight_for_positive_class)
# compute pos_weight = class_weights[1] / class_weights[0] if using sklearn balanced weights
# but easier: compute class counts
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
pos_weight = torch.tensor([neg / (pos + 1e-9)], dtype=torch.float32, device=device)  # >1 -> upweight positives
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

best_val_auc = 0.0  # We beginnen op 0
naam_van_bestand = "ultimate_clash_model.pt" # Kies een goede naam
save_path = os.path.join(MODEL_DIR, naam_van_bestand) 

print(f"💾 Het model wordt opgeslagen als: {save_path}")
for epoch in range(1, EPOCHS + 1):
    model.train()
    train_losses = []
    for batch in train_loader:
        p1_ids_b, p1_lvls_b, p2_ids_b, p2_lvls_b, y_b = batch
        p1_ids_b = p1_ids_b.to(device)
        p1_lvls_b = p1_lvls_b.to(device)
        p2_ids_b = p2_ids_b.to(device)
        p2_lvls_b = p2_lvls_b.to(device)
        y_b = y_b.to(device)

        optimizer.zero_grad()
        logits = model(p1_ids_b, p1_lvls_b, p2_ids_b, p2_lvls_b)
        loss = criterion(logits, y_b)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    # Validation
    model.eval()
    val_logits_all = []
    val_y_all = []
    val_losses = []
    with torch.no_grad():
        for batch in val_loader:
            p1_ids_b, p1_lvls_b, p2_ids_b, p2_lvls_b, y_b = batch
            p1_ids_b = p1_ids_b.to(device)
            p1_lvls_b = p1_lvls_b.to(device)
            p2_ids_b = p2_ids_b.to(device)
            p2_lvls_b = p2_lvls_b.to(device)
            y_b = y_b.to(device)

            logits = model(p1_ids_b, p1_lvls_b, p2_ids_b, p2_lvls_b)
            loss = criterion(logits, y_b)
            val_losses.append(loss.item())

            val_logits_all.append(logits.cpu().numpy())
            val_y_all.append(y_b.cpu().numpy())

    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)
    val_logits_all = np.concatenate(val_logits_all)
    val_y_all = np.concatenate(val_y_all)
    val_probs = 1 / (1 + np.exp(-val_logits_all))  # sigmoid
    val_auc = roc_auc_score(val_y_all, val_probs)
    val_pred = (val_probs >= 0.5).astype(int)
    val_acc = accuracy_score(val_y_all, val_pred)

    print(f"Epoch {epoch:02d}  train_loss={train_loss:.4f}  val_loss={val_loss:.4f}  val_auc={val_auc:.4f}  val_acc={val_acc:.4f}")

# Save best model by AUC
    if val_auc > best_val_auc:
        best_val_auc = val_auc
# We maken een 'koffer' met alles wat we later nodig hebben
        checkpoint_data = {
            "model_state": model.state_dict(),  # De hersenen (gewichten)
            "vocab_size": vocab_size,           # Instelling 1
            "max_level": max_level,             # Instelling 2
            "id_emb_dim": ID_EMBED_DIM,
            "lvl_emb_dim": LVL_EMBED_DIM,
            "hidden_dims": HIDDEN_DIMS,
            "dropout": DROPOUT,
            "num_heads": 4,                     # BELANGRIJK: Vergeet deze niet!
            "best_score": best_val_auc          # Handig om te weten hoe goed hij was
        }
        
        # Dit schrijft het daadwerkelijk naar je harde schijf
        torch.save(checkpoint_data, save_path)
        print("       💾 Opgeslagen op harde schijf.")
        
    else:
        print(f"   (Geen verbetering, we bewaren de oude versie met AUC {best_val_auc:.4f})")


💾 Het model wordt opgeslagen als: pytorch_card_model\ultimate_clash_model.pt
Epoch 01  train_loss=0.6709  val_loss=0.6581  val_auc=0.6403  val_acc=0.5957
       💾 Opgeslagen op harde schijf.
Epoch 02  train_loss=0.6596  val_loss=0.6528  val_auc=0.6525  val_acc=0.6054
       💾 Opgeslagen op harde schijf.
Epoch 03  train_loss=0.6561  val_loss=0.6509  val_auc=0.6592  val_acc=0.6114
       💾 Opgeslagen op harde schijf.
Epoch 04  train_loss=0.6541  val_loss=0.6494  val_auc=0.6624  val_acc=0.6143
       💾 Opgeslagen op harde schijf.
Epoch 05  train_loss=0.6527  val_loss=0.6479  val_auc=0.6647  val_acc=0.6153
       💾 Opgeslagen op harde schijf.
Epoch 06  train_loss=0.6516  val_loss=0.6462  val_auc=0.6663  val_acc=0.6165
       💾 Opgeslagen op harde schijf.
Epoch 07  train_loss=0.6509  val_loss=0.6460  val_auc=0.6675  val_acc=0.6180
       💾 Opgeslagen op harde schijf.
Epoch 08  train_loss=0.6502  val_loss=0.6449  val_auc=0.6691  val_acc=0.6184
       💾 Opgeslagen op harde schijf.
Epoch 09  t

In [54]:
# Cell 8 - Load model and prediction helper

ckpt = torch.load(os.path.join(MODEL_DIR, "best_model.pt"), map_location=device)
# instantiate a model with same hyperparams (from our variables)
loaded_model = TransformerCardNet(
    vocab_size=ckpt.get("vocab_size", vocab_size),
    max_level=ckpt.get("max_level", max_level),
    id_emb_dim=ID_EMBED_DIM,
    lvl_emb_dim=LVL_EMBED_DIM,
    hidden_dims=HIDDEN_DIMS,
    dropout=DROPOUT,
    num_heads=4  # <--- Vergeet deze niet, die is nieuw!
).to(device)
loaded_model.load_state_dict(ckpt["model_state"])
loaded_model.eval()

def predict_match(p1_ids, p1_lvls, p2_ids, p2_lvls, model=loaded_model):
    """
    p?_ids and p?_lvls are sequences/lists/arrays length 8 each (ints).
    Returns probability that player1 wins (float in [0,1]).
    """
    model.eval()
    with torch.no_grad():
        p1_ids_t = torch.tensor([p1_ids], dtype=torch.long, device=device)
        p1_lvls_t = torch.tensor([p1_lvls], dtype=torch.long, device=device)
        p2_ids_t = torch.tensor([p2_ids], dtype=torch.long, device=device)
        p2_lvls_t = torch.tensor([p2_lvls], dtype=torch.long, device=device)
        logit = model(p1_ids_t, p1_lvls_t, p2_ids_t, p2_lvls_t)
        prob = torch.sigmoid(logit).item()
    return prob

# quick test on a validation example
example_idx = 0
prob = predict_match(Xw_val_ids[example_idx], Xw_val_lvls[example_idx],
                     Xl_val_ids[example_idx], Xl_val_lvls[example_idx])
print("Pred prob player1 wins (example):", prob, "true label:", int(y_val[example_idx]))


RuntimeError: Error(s) in loading state_dict for TransformerCardNet:
	size mismatch for id_embed.weight: copying a param with shape torch.Size([102, 32]) from checkpoint, the shape in current model is torch.Size([102, 64]).
	size mismatch for transformer.layers.0.self_attn.in_proj_weight: copying a param with shape torch.Size([120, 40]) from checkpoint, the shape in current model is torch.Size([216, 72]).
	size mismatch for transformer.layers.0.self_attn.in_proj_bias: copying a param with shape torch.Size([120]) from checkpoint, the shape in current model is torch.Size([216]).
	size mismatch for transformer.layers.0.self_attn.out_proj.weight: copying a param with shape torch.Size([40, 40]) from checkpoint, the shape in current model is torch.Size([72, 72]).
	size mismatch for transformer.layers.0.self_attn.out_proj.bias: copying a param with shape torch.Size([40]) from checkpoint, the shape in current model is torch.Size([72]).
	size mismatch for transformer.layers.0.linear1.weight: co

In [55]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score
import os

# --- 1. BESTANDSNAMEN INSTELLEN ---
bestandsnaam_IN  = "ultimate_clash_model.pt"       # Het oude model (LEZEN)
bestandsnaam_UIT = "ultimate_clash_model_v2.pt"    # Het nieuwe model (SCHRIJVEN)

load_path = os.path.join(MODEL_DIR, bestandsnaam_IN)
save_path = os.path.join(MODEL_DIR, bestandsnaam_UIT)

EPOCHS_EXTRA = 5 
FINE_TUNE_LR = 0.0001 # Ik heb dit iets veiliger gezet (van 0.0005 naar 0.0001)

if 'train_loader' not in locals() or 'val_loader' not in locals():
    print("⚠️ FOUT: Geen data gevonden. Draai eerst je data-inlaad cel!")
else:
    if os.path.exists(load_path):
        print(f"📖 Lezen van: {bestandsnaam_IN}")
        print(f"✍️ Opslaan naar: {bestandsnaam_UIT}")
        
        # 1. LADEN
        checkpoint = torch.load(load_path, map_location=device)
        
        # Instellingen ophalen
        vocab = checkpoint.get("vocab_size", vocab_size) 
        heads = checkpoint.get("num_heads", 4)
        hidden = checkpoint.get("hidden_dims", HIDDEN_DIMS)
        dropout = checkpoint.get("dropout", DROPOUT)
        emb_id = checkpoint.get("id_emb_dim", ID_EMBED_DIM)
        emb_lvl = checkpoint.get("lvl_emb_dim", LVL_EMBED_DIM)
        m_lvl = checkpoint.get("max_level", max_level)
        
        # Beste score tot nu toe
        best_val_auc = checkpoint.get("best_score", 0.0)

        # Model bouwen
        model = TransformerCardNet(
            vocab_size=vocab, max_level=m_lvl, id_emb_dim=emb_id,
            lvl_emb_dim=emb_lvl, hidden_dims=hidden, dropout=dropout, num_heads=heads
        ).to(device)
        
        model.load_state_dict(checkpoint["model_state"])
        print(f"✅ Model ingeladen. Huidige lat ligt op AUC: {best_val_auc:.4f}")

        # 2. TRAINING SETUP
        optimizer = torch.optim.Adam(model.parameters(), lr=FINE_TUNE_LR) 
        criterion = nn.BCEWithLogitsLoss()
        
        print("🚀 Start training voor nieuwe versie...")
        
        for epoch in range(1, EPOCHS_EXTRA + 1):
            # --- FASE A: TRAINEN ---
            model.train()
            train_losses = []
            
            for batch in train_loader:
                p1_ids, p1_lvls, p2_ids, p2_lvls, y = batch
                p1_ids, p1_lvls = p1_ids.to(device), p1_lvls.to(device)
                p2_ids, p2_lvls = p2_ids.to(device), p2_lvls.to(device)
                y = y.to(device).float()
                
                optimizer.zero_grad()
                logits = model(p1_ids, p1_lvls, p2_ids, p2_lvls)
                loss = criterion(logits, y)
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())
            
            avg_train_loss = np.mean(train_losses)
            
            # --- FASE B: VALIDEREN ---
            model.eval()
            val_logits = []
            val_targets = []
            val_losses = [] # <--- DEZE WAS JE VERGETEN
            
            with torch.no_grad():
                for batch in val_loader:
                    p1_ids, p1_lvls, p2_ids, p2_lvls, y = batch
                    p1_ids, p1_lvls = p1_ids.to(device), p1_lvls.to(device)
                    p2_ids, p2_lvls = p2_ids.to(device), p2_lvls.to(device)
                    y = y.to(device).float()
                    
                    logits = model(p1_ids, p1_lvls, p2_ids, p2_lvls)
                    
                    # Ook hier loss berekenen
                    loss = criterion(logits, y) 
                    val_losses.append(loss.item())
                    
                    val_logits.append(logits.cpu().numpy())
                    val_targets.append(y.cpu().numpy())
            
            # --- FASE C: METRICS BEREKENEN ---
            # Alles samenvoegen
            val_logits_concat = np.concatenate(val_logits)
            val_y_concat = np.concatenate(val_targets)
            
            # Sigmoid voor kansen
            val_probs = 1 / (1 + np.exp(-val_logits_concat))
            
            # Scores berekenen
            current_auc = roc_auc_score(val_y_concat, val_probs)
            val_pred = (val_probs >= 0.5).astype(int)
            current_acc = accuracy_score(val_y_concat, val_pred)
            avg_val_loss = np.mean(val_losses)

            print(f"Epoch +{epoch}: Train Loss={avg_train_loss:.4f} | Val Loss={avg_val_loss:.4f} | Val AUC={current_auc:.4f} | Acc={current_acc:.4f}")

            # --- FASE D: OPSLAAN (Onder de NIEUWE naam) ---
            if current_auc > best_val_auc:
                print(f"   --> 🎉 Verbetering! (Was {best_val_auc:.4f}, nu {current_auc:.4f})")
                print(f"       Opgeslagen als '{bestandsnaam_UIT}'")
                best_val_auc = current_auc
                
                torch.save({
                    "model_state": model.state_dict(),
                    "vocab_size": vocab,
                    "num_heads": heads,
                    "max_level": m_lvl,
                    "hidden_dims": hidden,
                    "id_emb_dim": emb_id,
                    "lvl_emb_dim": emb_lvl,
                    "dropout": dropout,
                    "best_score": best_val_auc
                }, save_path) 
            else:
                print(f"   (Geen verbetering, we slaan niets op)")

        print(f"🏁 Klaar! Je origineel is veilig. De nieuwe versie (als hij beter was) heet: {bestandsnaam_UIT}")
        
    else:
        print(f"❌ Kan bestand '{bestandsnaam_IN}' niet vinden.")

📖 Lezen van: ultimate_clash_model.pt
✍️ Opslaan naar: ultimate_clash_model_v2.pt
✅ Model ingeladen. Huidige lat ligt op AUC: 0.6704
🚀 Start training voor nieuwe versie...
Epoch +1: Train Loss=0.6468 | Val Loss=0.6425 | Val AUC=0.6732 | Acc=0.6214
   --> 🎉 Verbetering! (Was 0.6704, nu 0.6732)
       Opgeslagen als 'ultimate_clash_model_v2.pt'
Epoch +2: Train Loss=0.6462 | Val Loss=0.6425 | Val AUC=0.6735 | Acc=0.6217
   --> 🎉 Verbetering! (Was 0.6732, nu 0.6735)
       Opgeslagen als 'ultimate_clash_model_v2.pt'
Epoch +3: Train Loss=0.6459 | Val Loss=0.6424 | Val AUC=0.6737 | Acc=0.6222
   --> 🎉 Verbetering! (Was 0.6735, nu 0.6737)
       Opgeslagen als 'ultimate_clash_model_v2.pt'
Epoch +4: Train Loss=0.6456 | Val Loss=0.6423 | Val AUC=0.6741 | Acc=0.6225
   --> 🎉 Verbetering! (Was 0.6737, nu 0.6741)
       Opgeslagen als 'ultimate_clash_model_v2.pt'
Epoch +5: Train Loss=0.6454 | Val Loss=0.6422 | Val AUC=0.6742 | Acc=0.6223
   --> 🎉 Verbetering! (Was 0.6741, nu 0.6742)
       Opgeslag